In [2]:
!pip install opencv-python mediapipe


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: C:\Users\DEVANSHI\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [9]:
import cv2
import mediapipe as mp
import random
import time

In [17]:
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(static_image_mode=False,
                       max_num_hands=1,
                       min_detection_confidence=0.7,
                       min_tracking_confidence=0.7)
mp_drawing = mp.solutions.drawing_utils


In [18]:
def count_fingers(hand_landmarks):
    tip_ids = [4, 8, 12, 16, 20]  # Thumb, Index, Middle, Ring, Pinky
    fingers = []

    # Thumb
    if hand_landmarks.landmark[tip_ids[0]].x < hand_landmarks.landmark[tip_ids[0] - 1].x:
        fingers.append(1)
    else:
        fingers.append(0)

    # 4 Fingers
    for i in range(1, 5):
        if hand_landmarks.landmark[tip_ids[i]].y < hand_landmarks.landmark[tip_ids[i] - 2].y:
            fingers.append(1)
        else:
            fingers.append(0)

    return sum(fingers)


In [19]:
def gesture_to_move(finger_count):
    if finger_count == 0:
        return "Rock"
    elif finger_count == 2:
        return "Scissors"
    elif finger_count == 5:
        return "Paper"
    else:
        return None

def get_winner(player, computer):
    if player == computer:
        return "Draw"
    elif (player == "Rock" and computer == "Scissors") or \
         (player == "Scissors" and computer == "Paper") or \
         (player == "Paper" and computer == "Rock"):
        return "You Win!"
    else:
        return "Computer Wins!"


In [ ]:
cap = cv2.VideoCapture(0)
game_started = False
player_move = None
result_text = ""
show_result = False
result_time = 0

print("Press 's' to start one round OR 'q' to quit")

while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame = cv2.flip(frame, 1)
    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    result = hands.process(rgb)

    key = cv2.waitKey(1) & 0xFF

    if key == ord('q'):
        break
    elif key == ord('s') and not game_started:
        game_started = True
        player_move = None
        show_result = False
        result_text = ""
        countdown_start = time.time()

    if game_started and time.time() - countdown_start < 3:
        cv2.putText(frame, f"Get ready... {3 - int(time.time() - countdown_start)}", (10, 50),
                    cv2.FONT_HERSHEY_COMPLEX, 1.0, (255, 0, 255), 4)
    elif game_started and player_move is None:
        if result.multi_hand_landmarks:
            for hand_landmarks in result.multi_hand_landmarks:
                mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)
                fingers_up = count_fingers(hand_landmarks)
                move = gesture_to_move(fingers_up)
                if move:
                    player_move = move
                    computer_move = random.choice(["Rock", "Paper", "Scissors"])
                    winner = get_winner(player_move, computer_move)
                    result_text = f"You:{player_move}| Computer:{computer_move}|{winner}"
                    show_result = True
                    result_time = time.time()
                    game_started = False

    if show_result:
        cv2.putText(frame, result_text, (10, 450),
                    cv2.FONT_HERSHEY_COMPLEX, 0.9, (255, 255, 0), 3)

        if time.time() - result_time > 4:
            show_result = False
            result_text = ""

    cv2.imshow("Rock Paper Scissors", frame)

cap.release()
cv2.destroyAllWindows()


Press 's' to start one round OR 'q' to quit
